<a href="https://colab.research.google.com/github/RSBalchII/cohort-15-data-science/blob/main/Project_2_Housing_Price_Prediction_v_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2: Housing Price Prediction


The project should be done using Regression prediction techniques ( with feature scaling and regularization ) as covered in class.
The goal is to minimize the RMS ***percentage*** error ( root mean squared percentage error - RMSPE ) on your prediction of the house sales price.



Any of the available features can be used in the regression, but a minimum set of variables that do the job should be the ultimate goal.



Be sure to go through the whole data science process and document as such in your Jupyter notebook.



This project will have less direct "To Do" guidance and the progression through the data science process will be more individualized this time around.
We will talk about all the issues during class so you're not going to be out on a ledge with this one, I just want to move you towards performing a data science project on your own eventually.



The data is available on AWS S3 at https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Data/Housing.Data.csv .


# Install Nvidia cudf dependencies and package

In [ ]:
%%capture
!pip install cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!apt-get update
!apt-get install -y nvidia-cuda-toolkit

import locale
locale.getpreferredencoding = lambda: "UTF-8"

%load_ext cudf.pandas

A data dictionary file is available at AWS S3 at [Housing Data Dictionary]( https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Housing%20-%20Data%20Documentation.pdf ).

## 1- Imports

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

from sklearn.impute import SimpleImputer

from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import StandardScaler


# 2- Loading the Data

In [3]:
# Load the data
url = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Data/Housing.Data.csv'
df = pd.read_csv(url)

# 3- Create Data Frame

In [4]:
df_1 = pd.DataFrame(df)

## Determine the "target"

In [5]:
# Identify the target variable (SalePrice)
target = df_1['SalePrice'].copy()

### Determining data distinctions

In [ ]:
# Identify categorical columns
categorical_cols = df_1.select_dtypes(include=['object', 'category']).columns.tolist()
print(f'Categorical columns{categorical_cols}')

# Identify numerical columns
numerical_cols = df_1.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f'\nNumerical columns{numerical_cols}')



### Asses Unique Values

In [ ]:
for col in categorical_cols:
    unique_values = df_1[col].unique()
    print(f"Unique values in column '{col}': {unique_values}")

### **Zacs**

In [8]:
# B =['Street']
# A = ['Alley']
# merged_df = pd.merge(
#     categorical_cols[A + B].value_counts(dropna = True).reset_index() ,
#     categorical_cols[B].value_counts(dropna = True).reset_index(),
#     on = B
#   )
# merged_df['marg_prob'] = merged_df['count_x']/merged_df['count_y'] * 100
# merged_df

### Mels


In [9]:
# pairs = ['Street', 'Heating']
# house_orig[pairs].value_counts()/house_orig[pairs].value_counts().sum()*100

### Minervas marginal Probability ex.

##### for col in categorical_cols:
```
for col in categorical_cols:
  print(categorical_cols[col].value_counts() /categorical_cols[col].notna().sum()*100 , '\n')
  ```

## Adapted marginal Probability for my setup

In [ ]:
# Print value counts and percentages for each categorical column
for col in categorical_cols:
    print(f"\nColumn: {col}")
    print(df[col].value_counts(normalize=True) * 100)


### Examination of categorical variables

In [ ]:
# Examine categorical variables
for col in categorical_cols:
    print(f"\nColumn: {col}")
    print(f"Number of unique values: {len(df_1[col].unique())}")
    print(f"Most common values:")
    print(df_1[col].value_counts(normalize=True).head())

    # Check if the column has too many unique values
    if len(df_1[col]) > 50:
        print(f"Warning: {col} has more than 50 unique values")

    # Check if the column is highly imbalanced
    if df_1[col].value_counts(normalize=True).max() > 0.95:
        print(f"Warning: {col} is highly imbalanced")


#

# 4- Data Cleaning

### isolate Float64 data columns

In [ ]:
# Get the names of the float columns
float_columns = df_1.select_dtypes(include=['float64']).columns

# Select the float columns from df_1
df_float = df_1[float_columns]

df_float_1 = df_float.copy()

df_float_1.describe()

### Cleaning null values

In [ ]:
# Check for missing values
print(df_float_1.isnull().sum())

### dropping Lot Frontage with high number of null values

In [ ]:
# Remove 'Lot Frontage' column
df_float_1 = df_float_1.drop('Lot Frontage', axis=1)

# Check for missing values
print(df_float_1.isnull().sum())

### Imputing data for the remainder of columns using looping method

In [ ]:
# Replace missing values with the mean of each column
for col in df_float_1.columns:
    df_float_1[col] = df_float_1[col].fillna(df_float_1[col].mean())

# Verify that there are no more missing values
print(df_float_1.isnull().sum())

In [16]:
features_for_correlation = df_float_1.copy()

### preparing cleaned feature data for feature correlation

In [ ]:
# Ensure SalePrice is in df_1
if 'SalePrice' not in df_1:
    print("SalePrice not found in df_1. Please check your data.")
else:
    # Concatenate df_float_1 with SalePrice
    features_for_correlation['SalePrice'] = df_1['SalePrice']

# Print the shape of the updated dataframe
print(f"Shape of df_float_1 after adding SalePrice: {features_for_correlation.shape}")

# Print the columns of df_float_1
print("Columns in features_for_correlation")
print(features_for_correlation.columns)

# 5a- Feature Selection

### Initial Correlation Visualization of remaining features

In [ ]:
# Calculate correlation matrix
corr_matrix = features_for_correlation.corr()

# Extract correlations with SalePrice
saleprice_corr = corr_matrix['SalePrice'].abs().sort_values(ascending=False)

# Print the top 10 most correlated features with SalePrice
print("Top 5 features correlated with SalePrice:")
print(saleprice_corr.head(10))

# Create a heatmap of the top 10 correlated features
top_features = saleprice_corr.index[:10].tolist()
top_features.append('SalePrice')

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix.loc[top_features, top_features], annot=True, cmap='coolwarm', square=True)
plt.title('Correlation Matrix of Top Features with SalePrice')
plt.show()

### Feature Selection based on correlation

In [ ]:
# Print the top 10 most correlated features with SalePrice
print("Top 10 features correlated with SalePrice:")
print(saleprice_corr.head(11))


In [20]:
# Select features with correlation > 0.5
selected_features = saleprice_corr[saleprice_corr > 0.5].index.tolist()

In [ ]:
# Remove 'SalePrice' from the list if it exists
if 'SalePrice' in selected_features:
    selected_features.remove('SalePrice')

# Print selected features
print("\nSelected features (correlation > 0.5):")
print(selected_features)

In [22]:
# Create a new dataframe with selected features
corr_features = features_for_correlation[selected_features].copy()

In [ ]:

# Print the shape of the new dataframe
print(f"\nShape of selected features dataframe: {corr_features.shape}")

# 5a1- Data Preparation

In [ ]:
# Print the shape of features and target
print(f"Features shape: {corr_features.shape}")
print(f"Target shape: {target.shape}")

### Splitting Training and Testing sets

In [25]:
# Data Preparation
X_train, X_test, y_train, y_test = train_test_split(corr_features, target, test_size=0.2, random_state=42)


In [ ]:
# Print the shapes of the split data
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

### Feature Scaling for corr_features

In [27]:
# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5a- Model Creation and Eval

In [ ]:
# Model Creation and Evaluation
# Use Ridge regression with a small alpha value to add some regularization
model = Ridge(alpha=0.1)
model.fit(X_train_scaled, y_train)

# Make predictions
predictions = model.predict(X_test_scaled)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"Root Mean Squared Error (RMSE): ${rmse:.2f}")

rmspe = np.sqrt(np.mean((predictions - y_test) / y_test)**2) * 100
print(f"Root Mean Squared Percentage Error (RMSPE): {rmspe:.2f}%")

r2 = model.score(X_test_scaled, y_test)
print(f"R-squared: {r2:.4f}")

# Visualize the predictions vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
plt.xlabel('Actual Price ($)')
plt.ylabel('Predicted Price ($)')
plt.title('Actual vs Predicted House Prices')
plt.show()


# 5b- RFE Feature Selection


### Feature Scaling for RFE Selection

In [ ]:
scaling_features = features_for_correlation.copy()
scaling_features.head()

In [30]:
X_train, X_test, y_train, y_test = train_test_split(scaling_features, target, test_size=0.2, random_state=42)

In [31]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#  Create a linear regression model
model = LinearRegression()

# Create an RFE object with 10 features to select
rfe = RFE(model, n_features_to_select=10)

# Fit the RFE object
rfe.fit(X_train, y_train)

# Get the selected features
selected_features_rfe = X_train.columns[rfe.support_]
print("Selected features by RFE:", selected_features_rfe)

# Get feature rankings
feature_rankings = rfe.ranking_
print("\nFeature rankings:")
for feature, rank in zip(X_train.columns, feature_rankings):
    print(f"{feature}: {rank}")


In [ ]:
# Initialize Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Create RFE object with 11 features to select
rfe = RFE(estimator=rf_regressor, n_features_to_select=11)

# Fit the RFE model
rfe.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[rfe.support_]
print("Selected Features:", selected_features)

# create a new Random Forest Regressor model using only the selected
new_rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the new model on the selected features
X_train_selected = X_train[selected_features]
new_rf_model.fit(X_train_selected, y_train)

# evaluate our new model:
# Make predictions on the test set
X_test_selected = X_test[selected_features]
y_pred = new_rf_model.predict(X_test_selected)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE of RFE-selected Random Forest Model: {rmse}")

# Visualize the predictions vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
plt.xlabel('Actual Price ($)')
plt.ylabel('Predicted Price ($)')
plt.title('Actual vs Predicted House Prices')
plt.show()


# 5c- RFECV Feature Selection

In [34]:
# # Create a linear regression model
# model = LinearRegression()

# # Create an RFECV object
# rfecv = RFECV(estimator=model, step=1, cv=5)

# # Fit the RFECV object
# rfecv.fit(X_train, y_train)

# # Get the selected features
# selected_features_rfecv = X_train.columns[rfecv.support_]
# print("\nSelected features by RFECV:", selected_features_rfecv)

# # Print the optimal number of features
# print(f"\nOptimal number of features: {rfecv.n_features_}")

# # Plot the cross-validation scores
# plt.figure(figsize=(10, 6))
# plt.xlabel("Number of features selected")
# plt.ylabel("Cross validation score")It
# plt.plot(range(1, len(rfecv.cv_results_['mean_test_score']) + 1), rfecv.cv_results_['mean_test_score'])
# plt.title("RFECV Cross Validation Scores")
# plt.show()
